In [1]:
import numpy as np
import openmm
import openmm.unit as unit
import openmm.app as app
from openmm import unit

from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange

import pysages
from pysages.grids import Grid
from pysages.colvars import DihedralAngle
from pysages.colvars.core import CollectiveVariable, multicomponent, build
from pysages.methods import ABF
from pysages.methods.utils import HistogramLogger
from pysages.approxfun import compute_mesh
from pysages.backends.openmm import get_cache
from utils import CVLogger
from openff.toolkit.topology import Molecule
from rdkit.Chem import Draw
import pickle

pi = np.pi

In [2]:
alanine_dipeptide = 'CC(=O)NC(C)C(=O)NC'
force_field = 'openff-2.0.0.offxml'

ala = Molecule.from_smiles(alanine_dipeptide, allow_undefined_stereo=True)
rdkit_mol = ala.to_rdkit()
for atom in rdkit_mol.GetAtoms():
    atom.SetProp('atomLabel', str(atom.GetIdx()))
img = Draw.MolToImage(rdkit_mol)
img.show()

ala.generate_conformers(n_conformers=1)
top = Topology.from_molecules(ala)
ff = ForceField(force_field)
inter = Interchange.from_smirnoff(force_field=ff, topology=top)

/home/xiangrui/miniconda3/envs/sim/lib/python3.12/site-packages/openff/amber_ff_ports/amber_ff_ports.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [3]:
cvs = [DihedralAngle([1,3,4,6]), DihedralAngle([3,4,6,8])]
timesteps = int(1e6)
bin_size = 32
grid = pysages.Grid(lower=(-pi, -pi), upper=(pi, pi), shape=(bin_size,bin_size), periodic=True)
prefix = f'results/'
filename = f'abf_{bin_size}'
output_stride = int(100)
callback = CVLogger(log_file=f'{prefix}{filename}', log_period=100)
method = ABF(cvs, grid, use_pinv=True)

In [4]:
def generate_simulation(inter=inter, file_name=prefix+filename, output_stride=output_stride):
    # log info
    trj_freq = output_stride
    data_freq = output_stride

    # Integration options
    time_step = 1 * unit.femtoseconds
    temperature = 300 * unit.kelvin
    friction = 10 / unit.picoseconds

    integrator = openmm.LangevinIntegrator(temperature, friction, time_step)
    simulation = inter.to_openmm_simulation(integrator=integrator)
    simulation.context.setVelocitiesToTemperature(temperature)
    simulation.context.setPeriodicBoxVectors(
        [40.0, 0.0, 0.0] * unit.angstrom,
        [0.0, 40.0, 0.0] * unit.angstrom,
        [0.0, 0.0, 40.0] * unit.angstrom
    )

    pdb_reporter = app.PDBReporter(f"{file_name}.pdb", trj_freq, enforcePeriodicBox=True)
    state_data_reporter = app.StateDataReporter(
        f"{file_name}.csv",
        data_freq,
        step=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation

In [5]:
results = pysages.run(method, generate_simulation, timesteps, callback)

In [6]:
with open(f'{prefix}{filename}.pkl', 'wb') as f:
    pickle.dump(results, f)